In [1]:
import netCDF4
import numpy as np
import pandas as pd
from datetime import datetime

In [2]:
def get_sci(path):
    # a function to get the sci flare list from the netcdf file
    # input: path to the netcdf file
    # output: a dataframe with the flare list: start time, end time, peak time, flare class, flare id
    sci = netCDF4.Dataset(path, 'r')
    # change it to the df
    xrsb_flux = sci.variables['xrsb_flux'][:].data
    status = sci.variables['status'][:]
    time = sci.variables['time'][:].data
    background_flux = sci.variables['background_flux'][:].data
    flare_class = sci.variables['flare_class'][:]
    itegrated_flux = sci.variables['integrated_flux'][:].data
    flare_id = sci.variables['flare_id'][:].data   # flare_counter, flare_id is the same

    sci_df = pd.DataFrame({'xrsb_flux': xrsb_flux, 'status': status, 'time': time, 
    'background_flux': background_flux, 'flare_class': flare_class, 'integrated_flux': itegrated_flux, 'flare_id': flare_id})

    # sort the df: one row for each flare
    start = []
    end = []
    peak = []
    cls = []
    IDs = []
    for id in sci_df['flare_id'].unique():
        fl = sci_df[sci_df['flare_id'] == id]
        try:
            fl_start = fl[fl['status'] =='EVENT_START']['time'].values[0]
            # time is the seconds since 2000-01-01 12:00:00[UTC], get the timestring in UTC
            fl_start = datetime.utcfromtimestamp(946728000 + fl_start).strftime('%Y-%m-%d %H:%M:%S')
        except:
            fl_start = None
        try:
            fl_end = fl[fl['status'] =='EVENT_END']['time'].values[0]
            fl_end = datetime.utcfromtimestamp(946728000 + fl_end).strftime('%Y-%m-%d %H:%M:%S')
        except:
            fl_end = None
            print(id)
        try:
            fl_peak = fl[fl['status'] =='EVENT_PEAK']['time'].values[0]
            fl_peak = datetime.utcfromtimestamp(946728000 + fl_peak).strftime('%Y-%m-%d %H:%M:%S')
        except:
            fl_peak = None
        try:
            fl_cls = fl[fl['status'] =='EVENT_PEAK']['flare_class'].values[0]
        except:
            fl_cls = None
        start.append(fl_start)
        end.append(fl_end)
        peak.append(fl_peak)
        cls.append(fl_cls)
        IDs.append(id)

    sci_flares = pd.DataFrame({'start time': start, 'end time': end, 'peak time': peak, 'fl_class': cls, 'id': IDs})
    # delte the rows with class = None
    sci_flares = sci_flares[sci_flares['fl_class'].notnull()]
    return sci_flares

In [3]:
go18 = get_sci("C:\\Users\\huke0\\Downloads\\sci_xrsf-l2-flsum_g18_s20220810_e20251113_v2-2-0.nc")

C:\Users\huke0\AppData\Local\Temp\ipykernel_20764\280928596.py:29: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  fl_start = datetime.utcfromtimestamp(946728000 + fl_start).strftime('%Y-%m-%d %H:%M:%S')
C:\Users\huke0\AppData\Local\Temp\ipykernel_20764\280928596.py:40: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  fl_peak = datetime.utcfromtimestamp(946728000 + fl_peak).strftime('%Y-%m-%d %H:%M:%S')
C:\Users\huke0\AppData\Local\Temp\ipykernel_20764\280928596.py:34: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to rep

202208100053
202208110036
202208110558
202208110631
202208132017
202208152020
202208262057
202209052015
202209090112
202209090740
202209100652
202209101917
202209102349
202209140330
202209162337
202209171242
202209181502
202209181518
202209182324
202209232150
202209232159
202209241422
202209252044
202209270408
202209271440
202209272253
202209281108
202209290514
202209291150
202209292346
202210010614
202210010754
202210021723
202210041226
202210050151
202210050611
202210050805
202210051031
202210051045
202210051055
202210051154
202210052011
202210090805
202210092344
202210140125
202210140920
202210170158
202210182341
202210200613
202210200625
202210220013
202210220034
202210220045
202210241314
202210270050
202210301435
202210310204
202211020627
202211061702
202211070641
202211100528
202211121428
202211121437
202211121626
202211121651
202211130158
202211150414
202211150428
202211152134
202211160314
202211160648
202211170726
202211171214
202211171517
202211190105
202211191235
202211201726

In [5]:
# only keep the flares after 2025-04-06
go18 = go18[go18['start time'] >= '2025-04-06 00:00:00'].reset_index(drop=True)

In [9]:
go18_location = get_location_df("C:\\Users\\huke0\\Downloads\\sci_xrsf-l2-flloc_g18_s20251031_e20251113_v2-2-0.nc")

In [10]:
# match the flare location with the flare 
go18 = go18.merge(go18_location, left_on='id', right_on='id', how='left')

In [7]:
go16 = get_sci("C:\\Users\\huke0\\Downloads\\sci_xrsf-l2-flsum_g16_s20170209_e20250406_v2-2-0.nc")

C:\Users\huke0\AppData\Local\Temp\ipykernel_22704\280928596.py:29: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  fl_start = datetime.utcfromtimestamp(946728000 + fl_start).strftime('%Y-%m-%d %H:%M:%S')
C:\Users\huke0\AppData\Local\Temp\ipykernel_22704\280928596.py:34: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  fl_end = datetime.utcfromtimestamp(946728000 + fl_end).strftime('%Y-%m-%d %H:%M:%S')
C:\Users\huke0\AppData\Local\Temp\ipykernel_22704\280928596.py:40: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to repre

201702090255
201702091052
201702091152
201702190858
201702210050
201702210508
201702210640
201702210920
201702211115
201702211126
201702211204
201702211348
201702211455
201702211909
201702212040
201702212241
201702221720
201702230040
201702240546
201702241529
201702270040
201702271510
201702271707
201702272252
201702272322
201702272348
201703020745
201703020811
201703030827
201703030838
201703032111
201703040127
201703040529
201703101013
201703101035
201703252317
201703261858
201703262258
201703262344
201703271459
201703271648
201703271753
201703271940
201703272153
201703282155
201703282343
201703290256
201703310715
201703310733
201703310924
201703311005
201704020220
201704020733
201704021040
201704021814
201704030050
201704030242
201704030418
201704030453
201704030516
201704030635
201704030809
201704030912
201704031001
201704031011
201704031106
201704031140
201704031202
201704031216
201704031300
201704031317
201704031417
201704031901
201704032005
201704032112
201704032159
201704032210

In [8]:
go16

,start time,end time,peak time,fl_class,id
0,2017-02-09 00:41:00,2017-02-09 00:57:00,2017-02-09 00:50:00,B3.8,201702090041
1,2017-02-09 01:30:00,2017-02-09 01:44:00,2017-02-09 01:40:00,B4.3,201702090130
2,2017-02-09 01:45:00,2017-02-09 01:56:00,2017-02-09 01:51:00,C1.7,201702090145
3,2017-02-09 02:31:00,2017-02-09 02:49:00,2017-02-09 02:40:00,B3.2,201702090231
4,2017-02-09 02:55:00,None,2017-02-09 02:59:00,B3.9,201702090255
...,...,...,...,...,...
14563,2025-04-05 23:46:00,2025-04-05 23:53:00,2025-04-05 23:49:00,C2.6,202504052346
14564,2025-04-06 10:35:00,2025-04-06 10:45:00,2025-04-06 10:41:00,C1.9,202504061035
14565,2025-04-06 15:46:00,2025-04-06 16:06:00,2025-04-06 15:53:00,C2.9,202504061546
14566,2025-04-06 17:41:00,2025-04-06 17:48:00,2025-04-06 17:44:00,C2.0,202504061741


In [8]:
def get_location_df(path):
    flloc = netCDF4.Dataset(path, 'r')
    flloc_hg = flloc.variables['flloc_hg'][:].data
    flloc_hg1 = flloc_hg[:, 0]
    flloc_hg2 = flloc_hg[:, 1]
    flloc_car = flloc.variables['flloc_car'][:].data
    flloc_car1 = flloc_car[:, 0]
    flloc_car2 = flloc_car[:, 1]
    flloc_rtheta = flloc.variables['flloc_rtheta'][:].data
    flloc_rtheta1 = flloc_rtheta[:, 0]
    flloc_rtheta2 = flloc_rtheta[:, 1]
    flloc_xy = flloc.variables['flloc_xy'][:].data
    flloc_xy1 = flloc_xy[:, 0]
    flloc_xy2 = flloc_xy[:, 1]
    flloc_solar_p_angle = flloc.variables['solar_p_angle'][:].data
    flloc_solar_radius = flloc.variables['solar_radius'][:].data
    flloc_id = flloc.variables['flare_id'][:].data

    flloc_df = pd.DataFrame({'hg1': flloc_hg1, 'hg2': flloc_hg2, 'car1': flloc_car1, 'car2': flloc_car2, 
    'rtheta1': flloc_rtheta1, 'rtheta2': flloc_rtheta2, 'xy1': flloc_xy1, 'xy2': flloc_xy2, 
    'solar_p_angle': flloc_solar_p_angle, 'solar_radius': flloc_solar_radius, 'id': flloc_id})
    return flloc_df

In [11]:
go16_location = get_location_df("C:\\Users\\huke0\\Downloads\\sci_xrsf-l2-flloc_g16_s20170209_e20250406_v2-2-0.nc")

In [13]:
# match the flare location with the flare 
go16 = go16.merge(go16_location, left_on='id', right_on='id', how='left')

In [21]:
go15 = get_sci("C:\\Users\\huke0\\Downloads\\sci_xrsf-l2-flsum_g15_s20100407_e20200304_v2-3-0 (1).nc")

C:\Users\huke0\AppData\Local\Temp\ipykernel_22704\280928596.py:29: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  fl_start = datetime.utcfromtimestamp(946728000 + fl_start).strftime('%Y-%m-%d %H:%M:%S')
C:\Users\huke0\AppData\Local\Temp\ipykernel_22704\280928596.py:40: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  fl_peak = datetime.utcfromtimestamp(946728000 + fl_peak).strftime('%Y-%m-%d %H:%M:%S')
C:\Users\huke0\AppData\Local\Temp\ipykernel_22704\280928596.py:34: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to rep

201004080232
201004170448
201004170501
201004170520
201004170535
201009031440
201009032234
201009040318
201009040353
201009051430
201009061600
201009062300
201009070506
201009070518
201009070840
201009082216
201009082233
201009082240
201009100032
201009102057
201009102253
201009150449
201009162347
201009190636
201009201911
201009201932
201009210717
201009220428
201009240325
201009250158
201009250213
201009262023
201009270007
201009272143
201009280147
201009281439
201009282200
201009282233
201009290917
201009300038
201009301701
201010010942
201010030416
201010041621
201010070914
201010072333
201010091739
201010131552
201010151350
201010152252
201010171023
201010171031
201010181523
201010181622
201010190232
201010192246
201010200700
201010201704
201010201715
201010212147
201010221338
201010241254
201010251549
201010251754
201010251843
201010252336
201010260001
201010261707
201010290107
201010290120
201010311347
201010311355
201011040231
201011042052
201011042156
201011042328
201011051241

In [22]:
go14 = get_sci("C:\\Users\\huke0\\Downloads\\sci_xrsf-l2-flsum_g14_s20090919_e20200304_v2-3-0 (1).nc")

C:\Users\huke0\AppData\Local\Temp\ipykernel_22704\280928596.py:29: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  fl_start = datetime.utcfromtimestamp(946728000 + fl_start).strftime('%Y-%m-%d %H:%M:%S')
C:\Users\huke0\AppData\Local\Temp\ipykernel_22704\280928596.py:34: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  fl_end = datetime.utcfromtimestamp(946728000 + fl_end).strftime('%Y-%m-%d %H:%M:%S')
C:\Users\huke0\AppData\Local\Temp\ipykernel_22704\280928596.py:40: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to repre

200912130905
200912140836
200912150431
200912151016
200912151242
200912151356
200912152050
200912152103
200912152111
200912170438
200912171500
200912171552
200912180108
200912181754
200912182017
200912182312
200912190132
200912191335
200912200742
200912200750
200912201537
200912210155
200912211025
200912211424
200912240240
200912310039
200912310327
200912310355
200912311045
201001031045
201001031852
201001031905
201001050840
201001070218
201001090729
201001101124
201001120509
201001171953
201001172208
201001181409
201001181953
201001190417
201001190810
201001190822
201001191258
201001191319
201001191839
201001200434
201001200645
201001200658
201001201018
201001201307
201001201314
201001201426
201001202135
201001202155
201001211651
201001241231
201001271224
201001271905
201001280740
201001280747
201002030447
201002061405
201002061417
201002070940
201002080222
201002080456
201002081042
201002081053
201002081330
201002081631
201002081820
201002082101
201002090009
201002092332
201002092358

In [23]:
go13 = get_sci("C:\\Users\\huke0\\Downloads\\sci_xrsf-l2-flsum_g13_s20130607_e20171214_v2-3-0.nc")

C:\Users\huke0\AppData\Local\Temp\ipykernel_22704\280928596.py:29: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  fl_start = datetime.utcfromtimestamp(946728000 + fl_start).strftime('%Y-%m-%d %H:%M:%S')
C:\Users\huke0\AppData\Local\Temp\ipykernel_22704\280928596.py:34: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  fl_end = datetime.utcfromtimestamp(946728000 + fl_end).strftime('%Y-%m-%d %H:%M:%S')
C:\Users\huke0\AppData\Local\Temp\ipykernel_22704\280928596.py:40: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to repre

201306072211
201306100021
201306100707
201306101920
201306121339
201306121928
201306131431
201306132349
201306132358
201306150339
201306151253
201306151303
201306162052
201306170305
201306181448
201306191641
201306210226
201306230834
201306252343
201306270743
201306271108
201308161210
201308220040
201308230000
201308231645
201308281606
201308292109
201309021035
201309021045
201309040407
201309041503
201309052158
201309071655
201309071711
201309101021
201309101031
201309121047
201309121628
201309132156
201309162039
201309170156
201309191546
201309192210
201309200411
201309200706
201309210131
201309210904
201309292142
201309292202
201309292233
201309292252
201310012010
201310021357
201310021951
201310022108
201310022113
201310022120
201310030407
201310031940
201310060011
201310070349
201310070717
201310071253
201310071529
201310082345
201310091801
201310102142
201310111429
201310111956
201310120545
201310121100
201310121449
201310121516
201310130839
201310140046
201310150116
201310150302

In [24]:
# go13 : 2016-05-03 13:00 - 2016-05-12 17:30
# go14 : 2010.1.1 - 2010.10.28 00:00, 2012-10-23 16:00 - 2012-11-19 16:31, 2015-05-21 18:00 - 2015-06-09 16:25, 2016-05-12 17:30 - 2016-06-09 17:30
# go15 : 2010-10-28 00:00 - 2012-10-2316:00, 2012-11-19 16:31 - 2015-05-2118:00, 2015-06-09 16:25 - 2016-05-03 13:00, 2016-06-09 17:30 - 2017-02-07 00:00
# go16 : 2017-02-07 00:00 - 2024-07-22 00:00


# go13
go13_ = go13[(go13['start time'] >= '2016-05-03 13:00') & (go13['start time'] < '2016-05-12 17:30')]
# go14
go14_1 = go14[(go14['start time'] >= '2010-01-01 00:00') & (go14['start time'] < '2010-10-28 00:00')]
go14_2 = go14[(go14['start time'] >= '2012-10-23 16:00') & (go14['start time'] < '2012-11-19 16:31')]
go14_3 = go14[(go14['start time'] >= '2015-05-21 18:00') & (go14['start time'] < '2015-06-09 16:25')]
go14_4 = go14[(go14['start time'] >= '2016-05-12 17:30') & (go14['start time'] < '2016-06-09 17:30')]
go14_ = pd.concat([go14_1, go14_2, go14_3, go14_4])
# go15
go15_1 = go15[(go15['start time'] >= '2010-10-28 00:00') & (go15['start time'] < '2012-10-23 16:00')]
go15_2 = go15[(go15['start time'] >= '2012-11-19 16:31') & (go15['start time'] < '2015-05-21 18:00')]
go15_3 = go15[(go15['start time'] >= '2015-06-09 16:25') & (go15['start time'] < '2016-05-03 13:00')]
go15_4 = go15[(go15['start time'] >= '2016-06-09 17:30') & (go15['start time'] < '2017-02-07 00:00')]
go15_ = pd.concat([go15_1, go15_2, go15_3, go15_4])

In [26]:
sci_20100101_20250406 = pd.concat([go13_, go14_, go15_, go16])

In [27]:
# check nan values
sci_20100101_20250406.isna().sum()

start time           5
end time          4321
peak time            0
fl_class             0
id                   0
hg1              20516
hg2              20516
car1             20516
car2             20516
rtheta1          20516
rtheta2          20516
xy1              20516
xy2              20516
solar_p_angle    20516
solar_radius     20516
dtype: int64

In [28]:
# delete the rows with nan start time
sci_20100101_20250406 = sci_20100101_20250406.dropna(subset=['start time'])

In [31]:
sci_20100101_20250406 = sci_20100101_20250406.reset_index(drop=True)

In [33]:
# reorder by start time
sci_20100101_20250406 = sci_20100101_20250406.sort_values(by=['start time']).reset_index(drop=True)

In [35]:
sci_20100101_20250406.to_csv("sci_20100101_20250406.csv", index=False)

In [12]:
sci_20100101_20250406 = pd.read_csv("sci_20100101_20250406.csv")

In [13]:
sci_20100101_20251113 = pd.concat([sci_20100101_20250406, go18])

In [15]:
sci_20100101_20251113.to_csv("sci_20100101_20251113.csv", index=False)